In [1]:
import sys
import os

# このセルが実行されるノートブックのあるディレクトリからプロジェクトルートを計算
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import numpy as np
from graph.prior.base import Prior
from core.uncertain_array import UncertainArray

class DummyPrior(Prior):
    def _compute_message(self, incoming: UncertainArray) -> UncertainArray:
        # 単純に精度を保ったゼロメッセージを返す
        return UncertainArray.zeros(self.shape, dtype=self.dtype, precision=incoming.to_scalar_precision())

In [3]:
X = DummyPrior(shape=(8, 8))  # X は Wave インスタンス
print(X)

Wave(shape=(8, 8), gen=1)


In [4]:
print(X.parent.generation)

0


In [5]:
# forward を呼んでメッセージを送信
X.parent.forward()

# Wave X が親からメッセージを受け取ったか確認
print("Parent message received:", X.parent_message)
print("Message type:", type(X.parent_message))

Parent message received: UncertainArray(shape=(8, 8), precision=scalar)
Message type: <class 'core.uncertain_array.UncertainArray'>


In [6]:
np.std(X.parent_message.data)

0.9902530254737346

In [7]:
X.parent.receive_message(X, UncertainArray.zeros(X.shape))

In [8]:
X.parent.forward()

In [9]:
print(X.parent_message.data)

[[0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]
 [0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j 0.+0.j]]


# Gaussian Prior

In [10]:
from graph.prior import GaussianPrior

X = GaussianPrior(mean=0, var=1, shape=(16, 16))
print(X)  # Wave instance
X.parent.forward()
print(X.parent_message)  # UA instance with zeros

Wave(shape=(16, 16), gen=1)
UncertainArray(shape=(16, 16), precision=scalar)


# Sparse Prior

In [11]:
import numpy as np
from graph.prior import SparsePrior
from core.uncertain_array import UncertainArray as UA
np.random.seed(1)

# スパース事前分布によるPriorを作成
X = SparsePrior(rho=0.2, shape=(2, 2), damping=0.0)
print("Wave created:", X)

# ---- 1回目の forward（output_message is None）----
X.parent.forward()
print("\n[Step 1] Initial random message")
print("Output message sent to Wave:", X.parent_message)
print("Belief (after first forward):", X.parent.belief)

# ---- 2回目：明示的に Prior にメッセージを送ってから forward ----
incoming = UA.random(shape=(2, 2), precision=1.0)  # よりシャープな観測を仮定
print(incoming.data)
X.parent.receive_message(X, incoming)
X.parent.forward()
print(X.parent_message.data)
print(X.parent_message.precision)
print("\n[Step 2] After receiving explicit message and forward")
print("Incoming message to Prior:", incoming)
print("New output message sent to Wave:", X.parent_message)
print("Updated belief (should reflect approximate posterior):", X.parent.belief)

Wave created: Wave(shape=(2, 2), gen=1)

[Step 1] Initial random message
Output message sent to Wave: UncertainArray(shape=(2, 2), precision=scalar)
Belief (after first forward): UncertainArray(shape=(2, 2), precision=scalar)
[[ 0.25857919+0.07176907j  0.43870724-0.53578489j]
 [ 0.14721498+0.48862675j -0.1883165 +0.16119851j]]
[[-0.00050253-0.00013948j -0.00678268+0.00828356j]
 [-0.00112374-0.00372984j  0.00031083-0.00026607j]]
[[15.89556733 11.08804485]
 [13.40461416 16.05181011]]

[Step 2] After receiving explicit message and forward
Incoming message to Prior: UncertainArray(shape=(2, 2), precision=scalar)
New output message sent to Wave: UncertainArray(shape=(2, 2), precision=array)
Updated belief (should reflect approximate posterior): UncertainArray(shape=(2, 2), precision=array)


In [12]:
import numpy as np
from graph.prior import SparsePrior
from core.uncertain_array import UncertainArray as UA

# 同じ seed を使って 2 つの Prior を生成
X1 = SparsePrior(rho=0.2, shape=(8, 8), damping=0.0, seed=42)
X2 = SparsePrior(rho=0.2, shape=(8, 8), damping=0.0, seed=42)

# 初回 forward（ランダムメッセージ生成）
X1.parent.forward()
X2.parent.forward()

# 比較
print("[Same Seed] Equal messages:", np.allclose(X1.parent_message.data, X2.parent_message.data))
print("[Same Seed] Equal belief:", np.allclose(X1.parent.belief.data, X2.parent.belief.data))

# 異なる seed を使ったケースも確認
X3 = SparsePrior(rho=0.2, shape=(8, 8), damping=0.0, seed=123)
X3.parent.forward()

print("[Different Seed] Different from seed=42:", not np.allclose(X1.parent_message.data, X3.parent_message.data))


[Same Seed] Equal messages: True
[Same Seed] Equal belief: True
[Different Seed] Different from seed=42: True
